In [ ]:
import pandas as pd

#Get dataframe
Constituent_Info = pd.read_csv('https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons.csv')
Constituent_Emails = pd.read_csv('https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email.csv')
Constituent_Subscription_Status = pd.read_csv('https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email_chapter_subscription.csv')

**People data frame**

In [ ]:
from numpy import NaN
## Check the general veracity of the data
## Make sure that all of email in the Constituent_Emails datafram are unique
len(Constituent_Emails.email.unique())
## Check to make sure the emails are not null, nan, NaN, or ''
# print(Constituent_Emails.email.value_counts()[NaN])

1400000

In [ ]:
#People table
#Becasue we have column is_unsub (Is the primary email address unsubscribed?) --> only get primary email
#Merger the first two data frame to get constituents' info and emails
#We want to keep all the emails so we do a right join (email df is on the right)
cons_info_email = pd.merge(Constituent_Info[['cons_id', 'create_dt', 'modified_dt', 'source']], Constituent_Emails[Constituent_Emails.is_primary == 1][['cons_id','email','cons_email_id']], how='right', on=['cons_id'])

In [ ]:
#Merge the obove data frame with the subscription status df to get constituents' info, email, and status.
#Becasue We only care about subscription statuses where chapter_id is 1 => filter subscription df to satisfy this condition
#If an email is not present in this table, it is assumed to still be subscribed where chapter_id is 1 => keep all emails => join on the opposite sign of the subscription df
cons_info_email_sub = pd.merge(Constituent_Subscription_Status[Constituent_Subscription_Status.chapter_id == 1][['cons_email_id', 'isunsub']], cons_info_email, how='right', on=['cons_email_id'])

In [ ]:
#Drop unrequired column:
people = cons_info_email_sub.drop(columns =['cons_id', 'cons_email_id'])

In [ ]:
#Check the data type of people data frame.
people.dtypes

isunsub        float64
create_dt       object
modified_dt     object
source          object
email           object
dtype: object

In [ ]:
# we can see this column has only 2 values: 1 and NaN
people['isunsub'].nunique()

2

In [ ]:
#change 1 to true , NaN to false:
people["isunsub"].fillna(False,inplace=True)
people.replace({'isunsub': {1.0: True}}, inplace = True)
people['isunsub'] = people['isunsub'].astype('bool')

In [ ]:
people['create_dt'] = people['create_dt'].astype('datetime64[ns]')
people['modified_dt'] = people['modified_dt'].astype('datetime64[ns]')

In [ ]:
# Change source and email to string
people["source"].fillna("Unknown",inplace=True)
people['source'] = people['source'].astype('string')
people['email'] = people['email'].astype('string')

In [ ]:
people

,isunsub,create_dt,modified_dt,source,email
0,False,1997-09-30 01:41:35,1981-02-26 19:36:22,Unknown,xmartinez@vincent.com
1,False,2014-03-27 23:18:18,2012-12-10 18:46:32,google,hmiller@haynes.biz
2,True,1992-06-01 06:07:45,1986-07-28 03:41:12,Unknown,aaron64@yahoo.com
3,True,1993-05-23 08:00:18,1983-05-07 09:29:18,Unknown,wyattvincent@hotmail.com
4,True,1986-10-31 03:24:05,1979-09-22 05:01:01,twitter,tspencer@hotmail.com
...,...,...,...,...,...
605634,False,1993-01-01 11:27:41,1978-11-25 23:47:50,google,smallmelvin@mitchell.com
605635,False,1979-02-07 03:11:36,1993-02-04 21:29:28,Unknown,gardnerchristian@hotmail.com
605636,False,2007-05-09 20:39:24,1993-05-10 02:58:28,google,ginanguyen@munoz.com
605637,False,1985-10-13 06:19:47,2019-02-17 20:19:53,organic,tatenicole@yahoo.com


In [ ]:
# Data types are now as required.
people.dtypes

isunsub                  bool
create_dt      datetime64[ns]
modified_dt    datetime64[ns]
source                 string
email                  string
dtype: object

In [ ]:
#Change column name and order as required:
people.rename(columns = {'email':'email', 'source':'code', 'isunsub':'is_unsub','create_dt':'created_dt','modified_dt':'updated_dt'}, inplace = True)
people = people[['email', 'code', 'is_unsub', 'created_dt', 'updated_dt']]

In [ ]:
people

,email,code,is_unsub,created_dt,updated_dt
0,xmartinez@vincent.com,Unknown,False,1997-09-30 01:41:35,1981-02-26 19:36:22
1,hmiller@haynes.biz,google,False,2014-03-27 23:18:18,2012-12-10 18:46:32
2,aaron64@yahoo.com,Unknown,True,1992-06-01 06:07:45,1986-07-28 03:41:12
3,wyattvincent@hotmail.com,Unknown,True,1993-05-23 08:00:18,1983-05-07 09:29:18
4,tspencer@hotmail.com,twitter,True,1986-10-31 03:24:05,1979-09-22 05:01:01
...,...,...,...,...,...
605634,smallmelvin@mitchell.com,google,False,1993-01-01 11:27:41,1978-11-25 23:47:50
605635,gardnerchristian@hotmail.com,Unknown,False,1979-02-07 03:11:36,1993-02-04 21:29:28
605636,ginanguyen@munoz.com,google,False,2007-05-09 20:39:24,1993-05-10 02:58:28
605637,tatenicole@yahoo.com,organic,False,1985-10-13 06:19:47,2019-02-17 20:19:53


In [ ]:
people.to_csv('people.csv', index = False)

**acquisition_facts table**

In [91]:
#Get only the created date from people df
acquisition_data = people[['created_dt']]
acquisition_data = people[['created_dt']].created_dt.dt.date

In [114]:
# Count the number of unique dates
acquisition_fact= acquisition_data.value_counts().reset_index().rename(columns = {"created_dt": 'acquisitions', "index": 'acquisition_date'})

In [115]:
acquisition_fact

,acquisition_date,acquisitions
0,1989-05-03,57
1,1997-03-11,56
2,1976-05-27,55
3,1987-08-29,55
4,2014-09-06,54
...,...,...
18440,1978-07-11,15
18441,1985-03-21,15
18442,2004-04-13,14
18443,2013-03-03,14


In [118]:
acquisition_fact.to_csv('acquisition_fact.csv', index = False)